In [ ]:
%%capture
!pip install transformers datasets librosa tqdm

In [ ]:
import torch
import librosa
from transformers import AutoProcessor, AutoModelForCTC
from datasets import load_dataset
import pandas as pd
from tqdm.notebook import tqdm
import random

In [ ]:
model_name = "kaarthu2003/wav2vec2-large-xls-r-53-telugu-final-2"
token = "insert_your_token_here"

In [ ]:
processor = AutoProcessor.from_pretrained(model_name, token=token)
model = AutoModelForCTC.from_pretrained(model_name, token=token)

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
def transcribe(audio):
    """
    Given an audio dictionary (expects a 'path' key),
    loads the audio file, resamples it to 16kHz,
    and returns the model's transcription.
    """
    # Load audio and resample to 16kHz
    audio_array, _ = librosa.load(audio["path"], sr=16000)
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

In [ ]:
dataset = load_dataset("openslr", "SLR66", trust_remote_code=True)

# Create train/test splits (here we work with the test split for evaluation)
seed = 42
splits = dataset["train"].train_test_split(test_size=0.2, seed=seed)
slr_test = splits["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/42.9k [00:00<?, ?B/s]

openslr.py:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4448 [00:00<?, ? examples/s]

In [ ]:
results = {"Actual Text": [], "Predicted Text": []}

In [ ]:
print(slr_test)

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 890
})


In [ ]:
random_indices = random.sample(range(len(slr_test)), 20)
random_examples = slr_test.select(random_indices)

In [ ]:
for example in tqdm(random_examples, desc="Evaluating"):
    # The ground truth transcription is assumed to be stored in the "text" field.
    actual_text = example["sentence"]

    # The "audio" field should be a dictionary containing a "path" to the audio file.
    predicted_text = transcribe(example["audio"])

    results["Actual Text"].append(actual_text)
    results["Predicted Text"].append(predicted_text)

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(results)
print(df)

                                          Actual Text  \
0         ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం   
1                                ది టైమ్స్ ఆఫ్ ఇండియా   
2   ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు...   
3             కటువుగా కైక వారికి నారచీరలు ఇప్పించింది   
4                                  ఆరు వందల అడుగుల్లో   
5                ఇందులో కొలెస్ట్రాల్ తక్కువగా ఉంటుంది   
6                      అంతరిక్ష నౌక రాకెట్ లో వాడతారు   
7     ఇక్కడ జిల్లా ప్రజా పరిషత్ ఉన్నత పాఠశాల ఉన్నది\n   
8                            వినదగు నెవ్వరు జెప్పిన\n   
9   ప్రతీ నిర్వాహకుడు వికీపీడియా తొలగింపు విధానం చ...   
10            అంగుళంలో భాగాలను భిన్నములుగా సూచిస్తారు   
11                 వారు వారి కుటుంబమంతా గర్వించింది\n   
12                              ఎప్పుడూ బిజీగా ఉంటాడు   
13  ఆమె జూలై పదహారు రెండు వేల తొమ్మిది న సహజంగా మర...   
14                   ఏదైనా మరో సారి మీకు నా కృతజ్ఞతలు   
15                     కాని ఐరావంతుడు ఒక షరతు పెడతాడు   
16                     మొండి వా

In [ ]:
from tabulate import tabulate
print(tabulate(df, headers="keys", tablefmt="pretty"))

+----+-------------------------------------------+----------------------------------------+
|    |                Actual Text                |             Predicted Text             |
+----+-------------------------------------------+----------------------------------------+
| 0  |        ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం        |      ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం       |
| 1  |               ది టైమ్స్ ఆఫ్ ఇండియా                |              ది టైమ్స్ ఆఫ్ ఇండియా              |
| 2  |   ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు వస్తున్నది   | ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు వస్తున్నది  |
| 3  |          కటువుగా కైక వారికి నారచీరలు ఇప్పించింది          |         కటువుగా కైకవారికి నారచీరలు ఇప్పించింది         |
| 4  |               ఆరు వందల అడుగుల్లో                |              ఆరు వందల అడుగుల్లో              |
| 5  |           ఇందులో కొలెస్ట్రాల్ తక్కువగా ఉంటుంది            |          ఇందులో కొలెస్ట్రాల్ తక్కోవగా వుంటుంది          |
| 6  | 

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

In [ ]:
# Load Whisper Small Telugu model & processor
processor_whisper = AutoProcessor.from_pretrained("kaarthu2003/whisper-small-telugu", token = token)
model_whisper = AutoModelForSpeechSeq2Seq.from_pretrained("kaarthu2003/whisper-small-telugu", token = token)

def transcribe_whisper(audio):
    """
    Given an audio dictionary (with a 'path' key),
    loads the audio file, resamples it to 16kHz,
    and returns the Whisper model's transcription.
    """
    # Load audio and resample to 16kHz
    audio_array, _ = librosa.load(audio["path"], sr=16000)
    # Process input for Whisper
    inputs = processor_whisper(audio_array, sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        generated_ids = model_whisper.generate(**inputs)
    # Decode the output
    predicted_text = processor_whisper.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return predicted_text

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

In [ ]:
# Prepare a dictionary for actual and predicted transcriptions
results_whisper = {"Actual Text": [], "Predicted Text": []}

# Evaluate Whisper on these 20 examples
for example in tqdm(random_examples, desc="Evaluating Whisper"):
    # Assuming the ground truth transcription is in "sentence"
    actual_text = example["sentence"]

    # Get the Whisper model's prediction
    predicted_text = transcribe_whisper(example["audio"])

    results_whisper["Actual Text"].append(actual_text)
    results_whisper["Predicted Text"].append(predicted_text)

Evaluating Whisper:   0%|          | 0/20 [00:00<?, ?it/s]

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
# Display results in a DataFrame
df_whisper = pd.DataFrame(results_whisper)
df_whisper

,Actual Text,Predicted Text
0,ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం,ఏమయినా అయోమయిని వృత్తి పేజీలను తయారు చెద్దాం
1,ది టైమ్స్ ఆఫ్ ఇండియా,ది టైమ్స్ ఆఫ్ ఇండియా
2,ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు...,ఆ చంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూర...
3,కటువుగా కైక వారికి నారచీరలు ఇప్పించింది,ఖటువుగా ఖైకవారికి నారచీరలు ఇపిపించింది
4,ఆరు వందల అడుగుల్లో,ఆరు వందల అడుగుల్లో
5,ఇందులో కొలెస్ట్రాల్ తక్కువగా ఉంటుంది,ఇందులో కొలెస్టరాల్ తక్కువగా వుంటుంది
6,అంతరిక్ష నౌక రాకెట్ లో వాడతారు,అంతరిక్ష నౌక రాకెట్ లో వాడతారు
7,ఇక్కడ జిల్లా ప్రజా పరిషత్ ఉన్నత పాఠశాల ఉన్నది\n,ఇక్కడ జిల్లా ప్రజా పరిషత్ ఉన్నత పాఠశాల ఉన్నది
8,వినదగు నెవ్వరు జెప్పిన\n,వినదగు నెవ్వరు జెప్పిన
9,ప్రతీ నిర్వాహకుడు వికీపీడియా తొలగింపు విధానం చ...,ప్రతీ నిర్వాహకుడు వికీపీడియా తొలగింపు విధానం చ...


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_whisper)

https://docs.google.com/spreadsheets/d/1sW5DM9AM1QRL1LjMaxM21vYDSJYb-qoudPS3D52R-rI#gid=0


In [ ]:
df

,Actual Text,Predicted Text
0,ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం,ఏమయినా అయోమయ నివృత్తి పేజీలను తయారు చేద్దాం
1,ది టైమ్స్ ఆఫ్ ఇండియా,ది టైమ్స్ ఆఫ్ ఇండియా
2,ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు...,ఆచంట మండలంలో వల్లూరు క్లిక్ చేస్తే కడప వల్లూరు...
3,కటువుగా కైక వారికి నారచీరలు ఇప్పించింది,కటువుగా కైకవారికి నారచీరలు ఇప్పించింది
4,ఆరు వందల అడుగుల్లో,ఆరు వందల అడుగుల్లో
5,ఇందులో కొలెస్ట్రాల్ తక్కువగా ఉంటుంది,ఇందులో కొలెస్ట్రాల్ తక్కోవగా వుంటుంది
6,అంతరిక్ష నౌక రాకెట్ లో వాడతారు,అంతరిక్ష నౌక రాకెట్లో వాడతారు
7,ఇక్కడ జిల్లా ప్రజా పరిషత్ ఉన్నత పాఠశాల ఉన్నది\n,ఇక్కడ జిల్లా ప్రజా పరిషత్ ఉన్నత పాఠశాల ఉన్నది
8,వినదగు నెవ్వరు జెప్పిన\n,వినదగు నెవ్వరు జెప్పిన
9,ప్రతీ నిర్వాహకుడు వికీపీడియా తొలగింపు విధానం చ...,ప్రతి నిర్వాహకుడు వికీపీడియా తొలగింపు విధానం చ...


No charts were generated by quickchart


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1b5l24Fz_1l0myXUrcafwolbGBCMc2YBozrC33tCU1TI#gid=0
